<a href="https://colab.research.google.com/github/kvinne-anc/Tanzania-Soil-Topography-Waterpumps-Project/blob/main/Tanz_Merged_Model_MVP_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git


Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-uifpu6we
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-uifpu6we
  Created wheel for geopandas: filename=geopandas-0.9.0+36.gcb88dd4-py2.py3-none-any.whl size=996489 sha256=e53a8d1e55725c343be7e2b2a7a8e5fe03266c791e5ecbb94dad5536e4a6e280
  Stored in directory: /tmp/pip-ephem-wheel-cache-gqgbrvgx/wheels/91/24/71/376c9c67192694168352

In [ ]:
!pip install category_encoders==2.*


In [ ]:
import pandas as pd
import geopandas as gdp 
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

In [ ]:
train = pd.merge(pd.read_csv('/content/final_waterpumps.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

test = pd.read_csv('/content/test_final.csv')

sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

FileNotFoundError: ignored

In [ ]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

In [ ]:
train['status_group'].value_counts(normalize=True)

functional                 0.543077
non functional             0.384238
functional needs repair    0.072685
Name: status_group, dtype: float64

In [ ]:
train


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,geometry,index_right,AREA,PERIMETER,SOIL_TYPE,MAIN_TYPE,status_group
43360,72938,0.0,2011-07-27,NaN,0,NaN,33.542898,-9.174777,Kwa Mzee Noa,0,Lake Nyasa,Mpandapanda,Mbeya,12,4,Rungwe,Kiwira,0,True,GeoData Consultants Ltd,VWC,K,NaN,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,POINT (33.54289809 -9.1747774),526.0,0.050868,1.190619,NTu1,Umbric Nitisols,functional
7263,65358,500.0,2011-03-23,Rc Church,2049,ACRA,34.665760,-9.308548,Kwa Yasinta Ng'Ande,0,Rufiji,Kitichi,Iringa,11,4,Njombe,Imalinyi,175,True,GeoData Consultants Ltd,WUA,Tove Mtwango gravity Scheme,True,2008,gravity,gravity,gravity,wua,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,POINT (34.66575986 -9.30854753),531.0,0.224534,2.967210,NTu5,Umbric Nitisols,functional
2486,469,25.0,2011-03-07,Donor,290,Do,38.238568,-6.179919,Kwasungwini,0,Wami / Ruvu,Kwedigongo,Pwani,6,1,Bagamoyo,Mbwewe,2300,True,GeoData Consultants Ltd,VWC,NaN,False,2010,india mark ii,india mark ii,handpump,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,POINT (38.23856758 -6.179919),290.0,0.579438,6.768834,CMo26,Ferralic Cambiso,functional
313,1298,0.0,2011-07-31,Government Of Tanzania,0,DWE,30.716727,-1.289055,Kwajovin 2,0,Lake Victoria,Kihanga,Kagera,18,1,Karagwe,Isingiro,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,other,other,other,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,POINT (30.71672742 -1.28905466),0.0,0.692150,6.306691,FRh16,Haplic Ferrasols,non functional
52726,27001,0.0,2011-03-10,Water,0,Gove,35.389331,-6.399942,Chama,0,Internal,Mtakuj,Dodoma,1,6,Bahi,Nondwa,0,True,GeoData Consultants Ltd,VWC,Zeje,True,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,POINT (35.3893315 -6.39994241),163.0,3.768181,42.348080,CMo9,Ferralic Cambiso,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,29534,50.0,2011-03-28,Wd And Id,489,WD and ID,38.268574,-5.450254,Kwa Maseko,4,Pangani,Komfungo A,Tanga,4,6,Handeni,Sindeni,1300,True,GeoData Consultants Ltd,VWC,WD and ID,False,2009,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,POINT (38.26857414 -5.45025371),214.0,0.510174,7.549393,FRr8,Rhodic Ferrasols,non functional
58170,20672,0.0,2011-12-07,Danida,0,Central government,33.926294,-9.641293,Kwa Asajile Mwangomale,0,Lake Nyasa,Mpanda,Mbeya,12,3,Kyela,Bujonde,0,True,GeoData Consultants Ltd,VWC,Ngana water supplied scheme,True,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,dry,dry,spring,spring,groundwater,communal standpipe,communal standpipe,POINT (33.92629443 -9.641292849999999),544.0,0.057141,1.216729,FLe1,Eutric Fluvisols,non functional
17191,10282,0.0,2013-02-02,World Bank,599,Nandra Construction,39.262924,-10.768079,Shuleni,0,Ruvuma / Southern Coast,Kilimahewa,Mtwara,90,33,Newala,Mchemo,1,True,GeoData Consultants Ltd,VWC,Malatu Chini,False,2012,submersible,submersible,submersible,vwc,user-group,other,

In [ ]:
#trained = train.drop(['extraction_type', 'id', 'index_right', 'quantity_group', 'num_private', 'basin', 'region', 'AREA', 'PERIMETER', 'district_code', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'extraction_type_group', 'extraction_type_class', 'management_group', 'management', 'payment', 'payment_type', 'quality_group', 'source_type', 'source_class', 'MAIN_TYPE', 'source', 'date_recorded', 'waterpoint_type_group'], axis=1)

In [ ]:
train = train.dropna()
train.isnull().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
s

In [ ]:
val = val.dropna()
train = train.dropna()
print(train.isnull().sum(), val.isnull().sum())

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
s

In [ ]:
X_train = train.drop('status_group', 1)
y_train = train['status_group']
X_val = val.drop('status_group', 1)
y_val = val['status_group']
X_test = test

In [ ]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
)


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['date_recorded', 'funder', 'installer',
                                      'wpt_name', 'basin', 'subvillage',
                                      'region', 'lga', 'ward', 'public_meeting',
                                      'recorded_by', 'scheme_management',
                                      'scheme_name', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_t...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='sqrt',
                               

In [ ]:
print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.8349785807412926


In [ ]:
# Predict on test
y_pred = pipeline.predict(X_test)

In [ ]:
Submission = pd.DataFrame(y_pred, columns = ['status_group'])
Submission

,status_group
0,functional
1,functional
2,functional
3,non functional
4,functional
...,...
14353,non functional
14354,non functional
14355,functional
14356,functional


In [ ]:
Submission['id'] = sample['id']
Submission = Submission[['id', 'status_group']]
Submission

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14353,39307,non functional
14354,18990,non functional
14355,28749,functional
14356,33492,functional


In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')
sample

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,functional
4,49871,functional
...,...,...
14353,39307,functional
14354,18990,functional
14355,28749,functional
14356,33492,functional


In [ ]:
Submission.to_csv('GeoTest6.csv', index=False)